# Quick Start

In order to use this .NET Interactive Notebook and play along with QuantLib (outside making your own app or plugging QuantLib into Quantower), you will need:

- Installed <a href="https://code.visualstudio.com/" target="_blank">Visual Studio Code</a>
- Installed <a href="https://dotnet.microsoft.com/download/dotnet/6.0" target="_blank">.NET 6 SDK</a>
- Installed <a href="https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode" target="_blank">.NET Interactive Notebooks</a> extension

For impatient, here is a simple example of calculating three moving averages (SMA, EMA and WMA) from 20 days of AAPL stock data using QuantLib (and YahooFinance to pull data), and then to chart it (using Plotly.NET):

In [ ]:
#r "nuget:YahooFinanceApi;"
#r "nuget:QuantLib;"
using YahooFinanceApi;
using QuantLib;

TSeries data = new();
var history = await Yahoo.GetHistoricalAsync("AAPL", DateTime.Today.AddDays(-20), DateTime.Now, Period.Daily);
SMA_Series sma = new(source: data, period: 5, useNaN: true);    // you can hide early data if desired
EMA_Series ema = new(source: data, period: 5);
WMA_Series wma = new(data, 5);

Console.Write($"date\t\t data\t SMA\t EMA\t WMA\t\n");
for (var i=0; i<history.Count-1; i++) {
    var item=history[i];
    data.Add((item.DateTime,  (double)item.Close));
    Console.Write($"{data[^1].t:yyyy-MM-dd}\t {(double)data:f2}\t {(double)sma:f2}\t {(double)ema:f2}\t {(double)wma:f2}\t\n");
}

date		 data	 SMA	 EMA	 WMA	
2022-03-07	 159.30	 NaN	 159.30	 159.30	
2022-03-08	 157.44	 NaN	 158.68	 158.06	
2022-03-09	 162.95	 NaN	 160.10	 160.50	
2022-03-10	 158.52	 NaN	 159.58	 159.71	
2022-03-11	 154.73	 158.59	 157.96	 158.05	
2022-03-14	 150.62	 156.85	 155.51	 155.39	
2022-03-15	 155.09	 156.38	 155.37	 154.81	
2022-03-16	 159.59	 155.71	 156.78	 155.88	
2022-03-17	 160.62	 156.13	 158.06	 157.51	
2022-03-18	 163.98	 157.98	 160.03	 160.13	
2022-03-21	 165.38	 160.93	 161.82	 162.60	
2022-03-22	 168.82	 163.68	 164.15	 165.23	
2022-03-23	 170.21	 165.80	 166.17	 167.40	
2022-03-24	 174.07	 168.49	 168.80	 170.16	
2022-03-25	 174.72	 170.64	 170.78	 172.24	


In [ ]:
#r "nuget: Plotly.NET, 2.0.0-preview.18"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.18"
using Plotly.NET;
using Plotly.NET.LayoutObjects;
GenericChart.GenericChart data_c = Chart2D.Chart.Line<DateTime,double,bool>(data.t, data.v, true,"data").WithLineStyle(Width: 2.5, Color: Color.fromString("blue"));
GenericChart.GenericChart sma_c = Chart2D.Chart.Line<DateTime,double,bool>(sma.t ,sma.v, true,"sma").WithLineStyle(Width: 1.5, Color: Color.fromString("red"));
GenericChart.GenericChart ema_c = Chart2D.Chart.Line<DateTime,double,bool>(ema.t,ema.v,true,"ema").WithLineStyle(Width: 1.5, Color: Color.fromString("green"));
GenericChart.GenericChart wma_c = Chart2D.Chart.Line<DateTime,double,bool>(wma.t,wma.v,true,"wma").WithLineStyle(Width: 1.5, Color: Color.fromString("purple"));
var chart = Chart.Combine(new []{data_c,sma_c,ema_c,wma_c}).WithSize(800,400).WithMargin(Margin.init<int, int, int, int, int, bool>(1,1,60,1,1,true));
chart

Installed Packages Plotly.NET, 2.0.0-preview.18 Plotly.NET.Interactive, 2.0.0-preview.18

Loading extensions from `Plotly.NET.Interactive.dll`

<!-- Plotly chart will be drawn inside this DIV -->

## Understanding QuantLib data model

QuantLib expects that every data item is a tuple (TimeDate t, double v) and TSeries is a list of (t,v) tuples. There are several helpers built into the TSeries class to simplify adding elements:

In [ ]:
var item1 = (DateTime.Today, 105.3);        // (DateTime, Value) tuple
double item2 = 293.1;                       // a simple double

TSeries data = new();
data.Add(item1);                            // adding tuple variable
data.Add(item2);                            // QuantLib stamps the (double) with current time
data.Add(0);                                // directly adding a number (stamped with current time)
data.Add((DateTime.Now.AddDays(-3), 10));   // adding a tuple with timestamp 3 days ago

data

index,Item1,Item2
0,2022-03-26 00:00:00Z,105.3
1,2022-03-26 21:09:19Z,293.1
2,2022-03-26 21:09:19Z,0
3,2022-03-23 21:09:19Z,10


TSeries list can display only values (without timestamps) or only timestamps (without values) by using `.v` or `.t` properties

In [ ]:
data.v

index,value
0,105.3
1,293.1
2,0
3,10


The last element on the list can be accessed by .Last() or by [^1] - and using `.t` (time) and `.v` (value) properties. Also, casting a TSeries into (double) will return the value of the last element

In [ ]:
bool IsTheSame = data.Last().v == data[^1].v;
double lastvalue = data;

lastvalue

10

Indicators are just smart TSeries classes; they get all required input during class construction (source of the datafeed, period...) and they automatically subscribe to events of the datafeed. Whenever datafeed gets a new value, indicator will calculate its own value. Indicators are also event publisers, so other indicators can subscribe to their results, chaining indicators together:

In [ ]:
TSeries t1 = new() {0,1,2,3,4,5,6,7,8,9}; // t1 is loaded with data and activated as a publisher
EMA_Series t2 = new(t1, 3);     // t2 will auto-load all history of t1 and wait for events from t1
ADD_Series t3 = new(t1, t2);    // t3 is an ADDition of t1 and t2 - will also load history and wait for t2 events
DIV_Series t4 = new(1, t3);     // t4 is calculating 1/t3 - and waiting for t3 events

TSeries t5 = new();             // a wild indicator appeared! And it is empty!
t4.Pub += t5.Sub;               // let us add a manual subscription to events coming from t4 - t5 is now listening to t4
t1.Add(0);                      // we add one new value to t1 - and trigger the full cascade of calculation! t5 is now full!

t5.v

index,value
0,Infinity
1,0.6666666666666666
2,0.3076923076923077
3,0.1951219512195122
4,0.1415929203539823
5,0.11072664359861592
6,0.09078014184397164
7,0.07687687687687687
8,0.06664931007550118
9,0.05881677197013211
